In [1]:
import checklist
from checklist.editor import Editor
import random
from lexicon import lexicons
import copy
from num2words import num2words
import csv
import os

In [2]:
lexicons.keys()

dict_keys(['MALE_NAME', 'FEMALE_NAME', 'NAME', 'TURN', 'DIRECTION', 'FRUIT', 'COLOUR', 'SCHOOL_ENTITY', 'SCHOOL_NOUN', 'EMOTION', 'KNOW', 'LOCATION', 'PROFESSION', 'ANIMAL', 'OBJECT_TABLE', 'ADJECTIVE_OF_PERSON', 'CITY', 'CONTINUOUS_VERB', 'SUBJECT', 'LOCATION_HOUSE', 'NEGATION_ADVERB', 'OBJECT', 'OBJECTS', 'N', 'YEAR', 'TIME', 'MILE', 'DOLLAR', 'T0_SYN_ANT', 'T1_POS_COM_SUP', 'T2_POS_COM_SUP_MISMATCH', 'T3_COM_ANTONYM', 'T4_COUNTRY_CAPITAL', 'T5_VV5O', 'T6_CAUSAL', 'T7_VERB_TENSE', 'T8_VERB_ASPECT', 'T9_VERB_WITH_OBJECT', 'T10_PROFESSION_ACTION', 'T11_IMPLICATURE', 'T12_RELATIONAL', 'T13_TAXONOMIC', 'COUNTRY', 'VERB'])

In [3]:
def Romanised(num):
    return num2words(int(num))

def Capitalise(word):
    return word[0].upper() + word[1:]

def AddCorrect(num1, num2):
    return str(int(num1) + int(num2))

def AddIncorrect(num1, num2):
    correct = int(num1) + int(num2)
    return str(random.choice([x for x in list(range(1,21)) if x != correct]))

def SubtractCorrect(num1, num2):
    return str(int(num1) - int(num2))

def SubtractIncorrect(num1, num2):
    correct = int(num1) - int(num2)
    return str(random.choice([x for x in list(range(1,21)) if x != correct]))

def RomanisedAddCorrect(num1, num2):
    return num2words(int(AddCorrect(num1, num2)))

def RomanisedAddIncorrect(num1, num2):
    return num2words(int(AddIncorrect(num1, num2)))

def RomanisedSubtractCorrect(num1, num2):
    return num2words(int(SubtractCorrect(num1, num2)))

def RomanisedSubtractIncorrect(num1, num2):
    return num2words(int(SubtractIncorrect(num1, num2)))

directions = ['North','East','South','West']

def DirectionCorrect(direction, turn):
    if turn == 'left':
        return directions[directions.index(direction) - 1]
    elif turn == 'right':
        return directions[directions.index(direction) - 3]
    else:
        return directions[directions.index(direction) - 2]
    
def DirectionIncorrect(direction, turn):
    correct = DirectionCorrect(direction, turn)
    return random.choice([dir for dir in directions if dir != correct])

def NumOtherThan(num1):
    return str(random.choice([x for x in range(6) if x != int(num1)]))

def NumLessThanInt(num1):
    return str(random.choice([x for x in range(6) if x < int(num1)]))

def NumLessThan(num1, key):
    return random.choice([x for i,x in enumerate(lexicons[key]) if i < lexicons[key].index(num1)])
    
func = {
    'Romanised':Romanised,
    'Capitalise':Capitalise,
    'AddCorrect':AddCorrect,
    'AddIncorrect':AddIncorrect,
    'SubtractCorrect':SubtractCorrect,
    'SubtractIncorrect':SubtractIncorrect,
    'RomanisedAddCorrect':RomanisedAddCorrect,
    'RomanisedAddIncorrect':RomanisedAddIncorrect,
    'RomanisedSubtractCorrect':RomanisedSubtractCorrect,
    'RomanisedSubtractIncorrect':RomanisedSubtractIncorrect,
    'DirectionCorrect':DirectionCorrect,
    'DirectionIncorrect':DirectionIncorrect,
    'NumOtherThan':NumOtherThan,
    'NumLessThanInt':NumLessThanInt,
    'NumLessThan':NumLessThan,
}

In [4]:
def make_template(premise, hypothesis, label):
    return {'P':premise,'H':hypothesis, 'label': label}

def save_file(examples, fname):
    with open(fname, 'w') as f:
        for ex in examples:
            f.write('%s\t%s\t%s\n'%(ex['label'],ex['P'],ex['H']))

def extract_placeholders(template):
    _list = []
    
    found = False
    curr = ''
    for i,ch in enumerate(template['P'] + ' ' + template['H']):
        if ch == '}':
            found = False
            _list.append(curr)
            curr = ''
        if found:
            curr += ch
        if ch == '{':
            found = True
    
    return _list

tuple_map = {
    'T0_SYN_ANT':['T0_ADJ','T0_SYN','T0_ANT'],
    'T1_POS_COM_SUP':['T1_POS','T1_COM','T1_SUP'],
    'T2_POS_COM_SUP_MISMATCH':['T2_POS','T2_COM','T2_SUP','T2_POS_MISMATCH','T2_COM_MISMATCH','T2_SUP_MISMATCH'],
    'T3_COM_ANTONYM':['T3_COM','T3_ANTONYM'],
    'T4_COUNTRY_CAPITAL':['T4_COUNTRY','T4_CAPITAL'],
    'T5_VV5O':['T5_VERB','T5_V5VERB','T5_OBJECT'],
    'T6_CAUSAL':['T6_CAUSAL_VERB','T6_CAUSAL_VERB_OPP','T6_CAUSAL_OBJECT'],
    'T7_VERB_TENSE':['T7_VERB_PAST','T7_VERB'],
    'T8_VERB_ASPECT':['T8_VERB_CONTINUOUS','T8_VERB'],
    'T9_VERB_WITH_OBJECT':['T9_PRESENT','T9_CONTINUOUS','T9_PAST'],
    'T10_PROFESSION_ACTION':['T10_PROFESSION','T10_ACTION'],
    'T11_IMPLICATURE':['T11_ACTION','T11_OBJECT'],
    'T12_RELATIONAL':['T12_RELATION','T12_RELATION_OPP','T12_RELATION_GENDER','T12_RELATION_MISMATCH'],
    'T13_TAXONOMIC':['T13_CLASS','T13_ELEMENT','T13_PROPERTY']
}

reverse_tuple_map = {
    
}

for key in tuple_map.keys():
    for elem in tuple_map[key]:
        reverse_tuple_map[elem] = key

def strip_index(placeholder):
    if placeholder[-1].isnumeric():
        return placeholder[:-1]
    else:
        return placeholder

def is_tuple(placeholder):
    if placeholder[0] == 'T' and (placeholder[2] == '_' or placeholder[3] == '_'):
        return True
    else:
        return False
    
def count_distinct(placeholders):
    
    _dict = {}
    
    def add_to_dict(ph):
        if strip_index(ph) not in _dict:
            _dict[strip_index(ph)] = []
        if ph not in _dict[strip_index(ph)]:
            _dict[strip_index(ph)].append(ph)
    
    for ph in placeholders:
        if ph == 'A' or ph.count('PRONOUN') > 0:
            continue
        elif ph.count('(') > 0:
            if ph.count('NumOtherThan') > 0 or ph.count('NumLessThanInt') > 0: continue
            args = ph[ph.find('(')+1:ph.find(')')]
            for ph1 in args.strip().split(','):
                ph1 = ph1.strip()
                add_to_dict(ph1)
        elif is_tuple(ph):
            ph = reverse_tuple_map[strip_index(ph)] + (ph[-1] if ph[-1].isnumeric() else '')
            add_to_dict(ph)
        else:
            add_to_dict(ph)
        
    return _dict

Ordered = ['N','MILE','TIME','YEAR','DOLLAR']

def sample(_dict):
    _fill = {}
    for key in _dict.keys():
        offset = 0
        if key in Ordered: 
            offset = 1
        sampled = random.sample(lexicons[key][offset:], len(_dict[key]))
        for i,x in enumerate(_dict[key]):
            _fill[x] = sampled[i]
    return _fill

def fill(_sampled, placeholders):
    _final_dict = {}
    for ph in placeholders:
        if ph.count('(') > 0:
            function = ph[:ph.find('(')]
            args = ph[ph.find('(')+1:ph.find(')')]    
            if function.count('Subtract') > 0:
                if (int(_sampled[args.split(',')[1]]) > int(_sampled[args.split(',')[0]])):
                    _sampled[args.split(',')[0]], _sampled[args.split(',')[1]] = _sampled[args.split(',')[1]], _sampled[args.split(',')[0]]
    for ph in placeholders:
        if ph == 'A': continue
        elif ph.count('PRONOUN') > 0:
            if ph == 'CAPITAL_PRONOUN':
                _final_dict[ph] = 'He' if _sampled['NAME' + ph[-1] if ph[-1].isnumeric() else 'NAME'] in lexicons['MALE_NAME'] else 'She'
            elif ph == 'PRONOUN':
                _final_dict[ph] = 'he' if _sampled['NAME' + ph[-1] if ph[-1].isnumeric() else 'NAME'] in lexicons['MALE_NAME'] else 'she'
            elif ph == 'POSSESSIVE_PRONOUN':
                _final_dict[ph] = 'his' if _sampled['NAME' + ph[-1] if ph[-1].isnumeric() else 'NAME'] in lexicons['MALE_NAME'] else 'her'
        elif ph.count('(') > 0:
            function = ph[:ph.find('(')]
            args = ph[ph.find('(')+1:ph.find(')')]
            if function == 'NumLessThan':
                _final_dict[ph] = func[function](_sampled[args], strip_index(args))
            elif function == 'NumLessThanInt' or function == 'NumOtherThan':
                _final_dict[ph] = func[function](args)
            elif args.count(',') > 0:
                _final_dict[ph] = func[function](_sampled[args.split(',')[0]],_sampled[args.split(',')[1]])
            else:
                _final_dict[ph] = func[function](_sampled[args])
        elif is_tuple(ph):
            phkey = reverse_tuple_map[strip_index(ph)] + (ph[-1] if ph[-1].isnumeric() else '')
            for i,tkey in enumerate(tuple_map[reverse_tuple_map[strip_index(ph)]]):
                if strip_index(ph) == tkey:
                    _final_dict[ph] = _sampled[phkey][i]
        else:
            _final_dict[ph] = _sampled[ph]
        
    return _final_dict

def apply_and_post_process(_final_dict, template):
    for ph in _final_dict.keys():
        template['P'] = template['P'].replace('{' + ph + '}', _final_dict[ph])
        template['H'] = template['H'].replace('{' + ph + '}', _final_dict[ph])
    while 1:
        if template['P'].count('{A}') > 0:
            idx = template['P'].find('{A}')
            template['P'] = template['P'].replace('{A}','an' if template['P'][idx + 4] in 'aeiou' else 'a', 1)
        elif template['H'].count('{A}') > 0:
            idx = template['H'].find('{A}')
            template['H'] = template['H'].replace('{A}','an' if template['H'][idx + 4] in 'aeiou' else 'a', 1)
        else:
            break
    return template

def fill_template(template):
    placeholders = extract_placeholders(template)
    _dict = count_distinct(placeholders)
    _sampled = sample(_dict)
    _final_dict = fill(_sampled, placeholders)
    filled_template = apply_and_post_process(_final_dict, template)
    return filled_template

In [5]:
T = {}

T['Lexical'] = [
    make_template('{NAME} is {T0_ADJ}.','{NAME} is {T0_SYN}.','entailment'),
    make_template('{NAME} is {T0_ADJ}.','{NAME} is {T0_ANT}.','contradiction'),
    make_template('{NAME} is {ADJECTIVE_OF_PERSON}.','{NAME} lives in {COUNTRY}.','neutral'),
    make_template('{NAME} is {ADJECTIVE_OF_PERSON}.','{NAME} is {A} {PROFESSION}.','neutral'),
    make_template('{NAME} lives in {COUNTRY}.','{NAME} is {ADJECTIVE_OF_PERSON}.','neutral'),
    make_template('{NAME} lives in {COUNTRY}.','{NAME} is {A} {PROFESSION}.','neutral'),
    make_template('{NAME} is {A} {PROFESSION}.','{NAME} lives in {COUNTRY}.','neutral'),
    make_template('{NAME} is {A} {PROFESSION}.','{NAME} is {ADJECTIVE_OF_PERSON}.','neutral'),
    make_template('{NAME} was born in {COUNTRY1}.','{NAME} is born in {COUNTRY2}.','contradiction'),
    make_template('{Capitalise(OBJECT)} is {COLOUR1} in colour.','{Capitalise(OBJECT)} is {COLOUR2} in colour.','contradiction'),
    make_template('That animal is {A} {ANIMAL1}.','That animal is {A} {ANIMAL2}.','contradiction'),
    make_template('{NAME} is eating {A} {FRUIT1}.','{NAME} is eating {A} {FRUIT2}.','contradiction')
]

T['Syntactic'] = [
    make_template('{NAME1} but not {NAME2} is {A} {PROFESSION}.','{NAME1} is {A} {PROFESSION}.','entailment'),
    
    make_template('Instead of {CONTINUOUS_VERB1} {NAME} is {CONTINUOUS_VERB2}.','{NAME} is {CONTINUOUS_VERB2}.','entailment'),
    make_template('Instead of {CONTINUOUS_VERB1} {NAME} is {CONTINUOUS_VERB2}.','{NAME} is {CONTINUOUS_VERB1}.','contradiction'),
    
    make_template('{NAME1} is {A} {PROFESSION} and {NAME2} is too.','{NAME2} is {A} {PROFESSION}.', 'entailment'),
    make_template('{NAME1} is {A} {PROFESSION} and {NAME2} is too.','{NAME2} is not {A} {PROFESSION}.', 'contradiction'),
    
    make_template('{NAME1} is {CONTINUOUS_VERB} and so is {NAME2}.','{NAME2} is {CONTINUOUS_VERB}.', 'entailment'),
    make_template('{NAME1} is {CONTINUOUS_VERB} and so is {NAME2}.','{NAME2} is not {CONTINUOUS_VERB}.', 'contradiction'),
    
    make_template('{NAME} tried but wasn\'t able to {VERB}.','{NAME} tried to {VERB}.','entailment'),
    make_template('{NAME} tried but wasn\'t able to {VERB}.','{NAME} didn\'t try to {VERB}.','contradiction'),
]

T['Negation'] = [
    make_template('{NAME} is {A} {PROFESSION}.','{NAME} is not {A} {PROFESSION}.','contradiction'),
    make_template('{NAME1} but not {NAME2} is {A} {PROFESSION}.','{NAME2} is {A} {PROFESSION}.','contradiction'),
    make_template('{NAME1} is {A} {PROFESSION} but {NAME2} is not.','{NAME2} is {A} {PROFESSION}.','contradiction'),
    make_template('{NAME} {T5_V5VERB} {T5_OBJECT}.','{NAME} does not {T5_VERB} {T5_OBJECT}.','contradiction'),
    make_template('{NAME} {T5_V5VERB} {T5_OBJECT}.','{NAME} never {T5_V5VERB} {T5_OBJECT}.','contradiction'),
    make_template('{NAME} {T5_V5VERB} {T5_OBJECT}.','{NAME} {NEGATION_ADVERB} {T5_VERB} {T5_OBJECT}.','contradiction'),
    make_template('Does {NAME} {T5_VERB} {T5_OBJECT}? No.','{NAME} {T5_V5VERB} {T5_OBJECT}.','contradiction'),
    make_template('Does {NAME} {T5_VERB} {T5_OBJECT}? No.','{NAME} does not {T5_VERB} {T5_OBJECT}.','entailment'),
]

T['Boolean'] = [
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.','{NAME} is {ADJECTIVE_OF_PERSON1}.','entailment'),
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.','{NAME} is {ADJECTIVE_OF_PERSON2}.','entailment'),
    
    make_template('{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME1} is {ADJECTIVE_OF_PERSON}.','entailment'),
    make_template('{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME2} is {ADJECTIVE_OF_PERSON}.','entailment'),
    
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1}, {ADJECTIVE_OF_PERSON2} and {ADJECTIVE_OF_PERSON3}.','{NAME} is {ADJECTIVE_OF_PERSON1}.','entailment'),
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1}, {ADJECTIVE_OF_PERSON2} and {ADJECTIVE_OF_PERSON3}.','{NAME} is {ADJECTIVE_OF_PERSON2}.','entailment'),
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1}, {ADJECTIVE_OF_PERSON2} and {ADJECTIVE_OF_PERSON3}.','{NAME} is {ADJECTIVE_OF_PERSON3}.','entailment'),
    
    make_template('{NAME1}, {NAME2} and {NAME3} are {ADJECTIVE_OF_PERSON}.','{NAME1} is {ADJECTIVE_OF_PERSON}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are {ADJECTIVE_OF_PERSON}.','{NAME2} is {ADJECTIVE_OF_PERSON}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are {ADJECTIVE_OF_PERSON}.','{NAME3} is {ADJECTIVE_OF_PERSON}.','entailment'),
    
    make_template('{NAME1} or {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME1} is {ADJECTIVE_OF_PERSON}.','neutral'),
    make_template('{NAME1} or {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME2} is {ADJECTIVE_OF_PERSON}.','neutral'),
    
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.','{NAME} is {ADJECTIVE_OF_PERSON2} and {ADJECTIVE_OF_PERSON1}.','entailment'),
    make_template('{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME2} and {NAME1} are {ADJECTIVE_OF_PERSON}.','entailment'),
    
    make_template('{NAME} is {ADJECTIVE_OF_PERSON1}. {NAME} is {ADJECTIVE_OF_PERSON2}.','{NAME} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.','entailment'),
    make_template('{NAME1} is {ADJECTIVE_OF_PERSON}. {NAME2} is {ADJECTIVE_OF_PERSON}.','{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON}.','entailment'),
    
    make_template('{NAME2} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.','{NAME} is {ADJECTIVE_OF_PERSON1}. {NAME} is {ADJECTIVE_OF_PERSON2}.','entailment'),
    make_template('{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON}.','{NAME1} is {ADJECTIVE_OF_PERSON}. {NAME2} is {ADJECTIVE_OF_PERSON}.','entailment'),
    
    make_template('{NAME1} is {ADJECTIVE_OF_PERSON1}. {NAME2} is {ADJECTIVE_OF_PERSON2}.','{NAME1} and {NAME2} are {ADJECTIVE_OF_PERSON1}.','neutral'),
    
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME1} is from {COUNTRY1}.','entailment'),
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME2} is from {COUNTRY2}.','entailment'),
    
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME1} is from {COUNTRY2}.','contradiction'),
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME2} is from {COUNTRY1}.','contradiction'),
    
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME1} is from {COUNTRY1}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME2} is from {COUNTRY2}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME3} is from {COUNTRY3}.','entailment'),
    
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME1} is from {COUNTRY2}.','contradiction'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME2} is from {COUNTRY3}.','contradiction'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME3} is from {COUNTRY1}.','contradiction'),
    
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME1} is from {COUNTRY1}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME2} is from {COUNTRY2}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME3} is from {COUNTRY3}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME4} is from {COUNTRY4}.','entailment'),
    
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME1} is from {COUNTRY2}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME2} is from {COUNTRY3}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME3} is from {COUNTRY4}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.','{NAME4} is from {COUNTRY1}.','contradiction'),
    
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME1} is from {COUNTRY1}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME2} is from {COUNTRY2}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME3} is from {COUNTRY3}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME4} is from {COUNTRY4}.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME5} is from {COUNTRY5}.','entailment'),
    
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME1} is from {COUNTRY2}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME2} is from {COUNTRY3}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME3} is from {COUNTRY4}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME4} is from {COUNTRY5}.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.','{NAME5} is from {COUNTRY1}.','contradiction'),
    
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME1} is not from {COUNTRY2}.','entailment'),
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME2} is not from {COUNTRY1}.','entailment'),
    
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME1} is not from {COUNTRY1}.','contradiction'),
    make_template('{NAME1} and {NAME2} are from {COUNTRY1} and {COUNTRY2} respectively.','{NAME2} is not from {COUNTRY2}.','contradiction'),
    
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME1} is not from {COUNTRY2}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME2} is not from {COUNTRY3}.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME3} is not from {COUNTRY1}.','entailment'),
    
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME1} is not from {COUNTRY1}.','contradiction'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME2} is not from {COUNTRY2}.','contradiction'),
    make_template('{NAME1}, {NAME2} and {NAME3} are from {COUNTRY1}, {COUNTRY2} and {COUNTRY3} respectively.','{NAME3} is not from {COUNTRY3}.','contradiction'),
    
    make_template('{NAME} is {A} {PROFESSION1} or {A} {PROFESSION2} but not both.','{NAME} is both {A} {PROFESSION1} and {A} {PROFESSION2}.','contradiction'),
    make_template('{NAME} is both {A} {PROFESSION1} and {A} {PROFESSION2}.','{NAME} is {A} {PROFESSION1} but not {A} {PROFESSION2}.','contradiction'),
    make_template('{NAME} is both {A} {PROFESSION1} and {A} {PROFESSION2}.','{NAME} is {A} {PROFESSION2} but not {A} {PROFESSION1}.','contradiction'),
    
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE1}.','{NAME} also asked for {OBJECT_TABLE2}.','neutral'),
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE2}.','{NAME} also asked for {OBJECT_TABLE1}.','neutral'),
]

T['Comparative'] = [
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME1} and {NAME2} can be compared in terms of how {T1_POS} they are.','entailment'),
    make_template('{NAME1} is {T2_COM} than {NAME2}.','{NAME1} and {NAME2} can be compared in terms of how {T2_POS_MISMATCH} they are.','neutral'),
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME2} is {T1_COM} than {NAME1}.','contradiction'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME2} is {T1_COM} than {NAME3}.','{NAME1} is {T1_COM} than {NAME2}.','entailment'),
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME2} is {T1_COM} than {NAME3}.','{NAME2} is {T1_COM} than {NAME3}.','entailment'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME2} is {T1_COM} than {NAME3}.','{NAME1} is {T1_COM} than {NAME3}.','entailment'),
    make_template('{NAME2} is {T1_COM} than {NAME3}. {NAME1} is {T1_COM} than {NAME2}.','{NAME1} is {T1_COM} than {NAME3}.','entailment'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME2} is {T1_COM} than {NAME3}.','{NAME3} is {T1_COM} than {NAME1}.','contradiction'),
    make_template('{NAME2} is {T1_COM} than {NAME3}. {NAME1} is {T1_COM} than {NAME2}.','{NAME3} is {T1_COM} than {NAME1}.','contradiction'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.','{NAME2} is {T1_COM} than {NAME3}.','neutral'),
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.','{NAME3} is {T1_COM} than {NAME2}.','neutral'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME1}.','{NAME2} is {T1_COM} than {NAME3}.','neutral'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME1}.','{NAME3} is {T1_COM} than {NAME2}.','neutral'),
    
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME1} is {T1_COM} than {NAME2}.','entailment'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME1} is {T1_COM} than {NAME3}.','entailment'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME2} is {T1_COM} than {NAME1}.','entailment'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME2} is {T1_COM} than {NAME3}.','entailment'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME3} is {T1_COM} than {NAME1}.','entailment'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME3} is {T1_COM} than {NAME2}.','entailment'),
    
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME2} is {T1_COM} than {NAME3}.','neutral'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME3} is {T1_COM} than {NAME2}.','neutral'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME1} is {T1_COM} than {NAME3}.','neutral'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME3} is {T1_COM} than {NAME1}.','neutral'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME1} is {T1_COM} than {NAME2}.','neutral'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME2} is {T1_COM} than {NAME1}.','neutral'),
    
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME2} is {T1_COM} than {NAME1}.','contradiction'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','{NAME3} is {T1_COM} than {NAME1}.','contradiction'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME1} is {T1_COM} than {NAME2}.','contradiction'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','{NAME3} is {T1_COM} than {NAME2}.','contradiction'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME1} is {T1_COM} than {NAME3}.','contradiction'),
    make_template('Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','{NAME2} is {T1_COM} than {NAME3}.','contradiction'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','entailment'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME2} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','entailment'),
    make_template('{NAME3} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME2}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','entailment'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','contradiction'),
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','contradiction'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME2} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','contradiction'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME2} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','contradiction'),
    make_template('{NAME3} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME2}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','contradiction'),
    make_template('{NAME3} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME2}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','contradiction'),
    
    make_template('{NAME1} is {T1_COM} than {NAME3}. {NAME2} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','neutral'),
    make_template('{NAME1} is {T1_COM} than {NAME3}. {NAME2} is {T1_COM} than {NAME3}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','neutral'),
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME3} is {T1_COM} than {NAME2}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME1}.','neutral'),
    make_template('{NAME1} is {T1_COM} than {NAME2}. {NAME3} is {T1_COM} than {NAME2}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','neutral'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME1}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.','neutral'),
    make_template('{NAME2} is {T1_COM} than {NAME1}. {NAME3} is {T1_COM} than {NAME1}.','Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.','neutral'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME1} is atleast as {T1_POS} than {NAME2}.','entailment'),
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME2} is atleast as {T1_POS} than {NAME1}.','contradiction'),
    
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME1} is atmost as {T1_POS} than {NAME2}.','contradiction'),
    make_template('{NAME1} is {T1_COM} than {NAME2}.','{NAME2} is atmost as {T1_POS} than {NAME1}.','entailment'),
    
    make_template('{NAME1} is {T1_POS}. {NAME2} is {T1_COM}.','{NAME1} is {T1_COM} than {NAME2}.','contradiction'),
    make_template('{NAME1} is {T1_POS}. {NAME2} is {T1_COM}.','{NAME2} is {T1_COM} than {NAME1}.','entailment'),

    make_template('{NAME1} is {T3_COM} than {NAME2}.','{NAME2} is {T3_ANTONYM} than {NAME1}.','entailment'),
    make_template('{NAME1} is {T3_COM} than {NAME2} but not {NAME3}.','{NAME1} is {T3_ANTONYM} than {NAME3}.','entailment'),
]

T['Quantifier'] = [
    make_template('None of the {OBJECTS} are {COLOUR} in colour.','Some of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('None of the {OBJECTS} are {COLOUR} in colour.','All of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('Some of the {OBJECTS} are {COLOUR} in colour.','None of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('Some of the {OBJECTS} are {COLOUR} in colour.','All of the {OBJECTS} are {COLOUR} in colour.','neutral'),
    make_template('All of the {OBJECTS} are {COLOUR} in colour.','None of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('All of the {OBJECTS} are {COLOUR} in colour.','Some of the {OBJECTS} are {COLOUR} in colour.','entailment'),
    
    make_template('None of the {OBJECTS} are {COLOUR1} in colour.','Some of the {OBJECTS} are {COLOUR2} in colour.','neutral'),
    make_template('None of the {OBJECTS} are {COLOUR1} in colour.','All of the {OBJECTS} are {COLOUR2} in colour.','neutral'),
    make_template('Some of the {OBJECTS} are {COLOUR1} in colour.','None of the {OBJECTS} are {COLOUR2} in colour.','neutral'),
    make_template('Some of the {OBJECTS} are {COLOUR1} in colour.','All of the {OBJECTS} are {COLOUR2} in colour.','contradiction'),
    make_template('All of the {OBJECTS} are {COLOUR1} in colour.','None of the {OBJECTS} are {COLOUR2} in colour.','entailment'),
    make_template('All of the {OBJECTS} are {COLOUR1} in colour.','Some of the {OBJECTS} are {COLOUR2} in colour.','contradiction'),
    
    make_template('{NAME} has {N} {OBJECTS}.','{NAME} has atleast {NumLessThan(N)} {OBJECTS}.', 'entailment'),
    make_template('{NAME} has {NumLessThan(N)} {OBJECTS}.','{NAME} has atmost {N} {OBJECTS}.', 'entailment'),
    
    make_template('{NAME} has {N} {OBJECTS}.','{NAME} has atmost {NumLessThan(N)} {OBJECTS}.', 'contradiction'),
    make_template('{NAME} has {NumLessThan(N)} {OBJECTS}.','{NAME} has atleast {N} {OBJECTS}.', 'contradiction'),
]

T['Conditional'] = [
    make_template('If {NAME1} comes to the {LOCATION}, {NAME2} won\'t come. {NAME1} came to the {LOCATION}.','{NAME2} didn\'t come to the {LOCATION}.','entailment'),
    make_template('If {NAME1} comes to the {LOCATION}, {NAME2} won\'t come. {NAME1} did not come to the {LOCATION}.','{NAME2} didn\'t come to the {LOCATION}.','neutral'),
    make_template('{NAME1} will come to the {LOCATION} if {NAME2} comes. {NAME2} came to the {LOCATION}.','{NAME1} came to the {LOCATION}.','entailment'),
    
    make_template('If no one buys the {OBJECT}, {NAME1} will buy it. {NAME2} bought the {OBJECT}.','{NAME1} bought the {OBJECT}.','contradiction'),
    
    make_template('If {NAME} passes all the subjects, {PRONOUN} will get the scholarship. {NAME} failed {SUBJECT}.','{NAME} got the scholarship.','contradiction'),
    make_template('If {NAME} passes all the subjects, {PRONOUN} will get the scholarship. {NAME} failed {SUBJECT}.','{NAME} did not get the scholarship.','entailment'),
]

T['Numerical'] = [
    make_template('{NAME} has {N1} dollars. {CAPITAL_PRONOUN} received {N2} more dollars.','{NAME} now has {AddCorrect(N1,N2)} dollars.','entailment'),
    make_template('{NAME} has {N1} dollars. {CAPITAL_PRONOUN} received {N2} more dollars.','{NAME} now has {AddIncorrect(N1,N2)} dollars.','contradiction'),
    
    make_template('{NAME} has {Romanised(N1)} dollars. {CAPITAL_PRONOUN} received {Romanised(N2)} more dollars.','{NAME} now has {RomanisedAddCorrect(N1,N2)} dollars.','entailment'),
    make_template('{NAME} has {Romanised(N1)} dollars. {CAPITAL_PRONOUN} received {Romanised(N2)} more dollars.','{NAME} now has {RomanisedAddIncorrect(N1,N2)} dollars.','contradiction'),
    
    make_template('{NAME} has {N1} dollars. {CAPITAL_PRONOUN} gave away {N2} dollars.','{NAME} now has {SubtractCorrect(N1,N2)} dollars.','entailment'),
    make_template('{NAME} has {N1} dollars. {CAPITAL_PRONOUN} gave away {N2} dollars.','{NAME} now has {SubtractIncorrect(N1,N2)} dollars.','contradiction'),
    
    make_template('{NAME} has {Romanised(N1)} dollars. {CAPITAL_PRONOUN} gave away {Romanised(N2)} dollars.','{NAME} now has {RomanisedSubtractCorrect(N1,N2)} dollars.','entailment'),
    make_template('{NAME} has {Romanised(N1)} dollars. {CAPITAL_PRONOUN} gave away {Romanised(N2)} dollars.','{NAME} now has {RomanisedSubtractIncorrect(N1,N2)} dollars.','contradiction'),
    
    make_template('{NAME1} is the only child of {NAME0}.','{NAME0} has exactly 1 child.','entailment'),
    make_template('{NAME1} and {NAME2} are the only children of {NAME0}.','{NAME0} has exactly 2 children.','entailment'),
    make_template('{NAME1}, {NAME2} and {NAME3} are the only children of {NAME0}.','{NAME0} has exactly 3 children.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are the only children of {NAME0}.','{NAME0} has exactly 4 children.','entailment'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are the only children of {NAME0}.','{NAME0} has exactly 5 children.','entailment'),
    
    make_template('{NAME1} is the only child of {NAME0}.','{NAME0} has exactly {NumOtherThan(1)} child.','contradiction'),
    make_template('{NAME1} and {NAME2} are the only children of {NAME0}.','{NAME0} has exactly {NumOtherThan(2)} children.','contradiction'),
    make_template('{NAME1}, {NAME2} and {NAME3} are the only children of {NAME0}.','{NAME0} has exactly {NumOtherThan(3)} children.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3} and {NAME4} are the only children of {NAME0}.','{NAME0} has exactly {NumOtherThan(4)} children.','contradiction'),
    make_template('{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are the only children of {NAME0}.','{NAME0} has exactly {NumOtherThan(5)} children.','contradiction'),
    
    make_template('{NAME1} is the child of {NAME0}.','{NAME0} has atleast 1 child.','entailment'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}.','{NAME0} has atleast 2 children.','entailment'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}.','{NAME0} has atleast 3 children.','entailment'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}. {NAME4} is the child of {NAME0}.','{NAME0} has atleast 4 children.','entailment'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}. {NAME4} is the child of {NAME0}. {NAME5} is the child of {NAME0}.','{NAME0} has atleast 5 children.','entailment'),
    
    make_template('{NAME1} is the child of {NAME0}.','{NAME0} has no child.','contradiction'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}.','{NAME0} has only {NumLessThanInt(2)} children.','contradiction'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}.','{NAME0} has only {NumLessThanInt(3)} children.','contradiction'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}. {NAME4} is the child of {NAME0}.','{NAME0} has only {NumLessThanInt(4)} children.','contradiction'),
    make_template('{NAME1} is the child of {NAME0}. {NAME2} is the child of {NAME0}. {NAME3} is the child of {NAME0}. {NAME4} is the child of {NAME0}. {NAME5} is the child of {NAME0}.','{NAME0} has only {NumLessThanInt(5)} children.','contradiction'),
    
    make_template('{NAME1} has {N} coins. {NAME2} has {NumLessThan(N)} coins.','{NAME1} has more coins than {NAME2}.','entailment'),
    make_template('{NAME1} has {N} coins. {NAME2} has {NumLessThan(N)} coins.','{NAME2} has less coins than {NAME1}.','entailment'),
    
    make_template('{NAME1} has {NumLessThan(N)} coins. {NAME2} has {N} coins.','{NAME1} has more coins than {NAME2}.','contradiction'),
    make_template('{NAME1} has {NumLessThan(N)} coins. {NAME2} has {N} coins.','{NAME2} has less coins than {NAME1}.','contradiction'),
]

T['Temporal'] = [
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} was born after {NAME2}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} was born after {NAME1}.','entailment'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} was born before {NAME1}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} was born before {NAME2}.','entailment'),
    
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} was born before {NAME2}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} was born before {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} was born after {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} was born after {NAME2}.','contradiction'),
    
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} is younger than {NAME2}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} is younger than {NAME1}.','entailment'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} is elder than {NAME1}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} is elder than {NAME2}.','entailment'),
    
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} is elder than {NAME2}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} is elder than {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} is younger than {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} is younger than {NAME2}.','contradiction'),
    
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} was born later than {NAME2}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} was born later than {NAME1}.','entailment'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} was born earlier than {NAME1}.','entailment'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} was born earlier than {NAME2}.','entailment'),
    
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME1} was born earlier than {NAME2}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME2} was born earlier than {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.','{NAME2} was born later than {NAME1}.','contradiction'),
    make_template('{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.','{NAME1} was born later than {NAME2}.','contradiction'),
    
    make_template('{NAME1} reached {LOCATION} at {TIME} whereas {NAME2} reached at {NumLessThan(TIME)}.','{NAME1} reached after {NAME2}.','entailment'),
    make_template('{NAME1} reached {LOCATION} at {NumLessThan(TIME)} whereas {NAME2} reached at {TIME}.','{NAME1} reached before {NAME2}.','entailment'),
    
    make_template('{NAME1} reached {LOCATION} at {TIME} whereas {NAME2} reached at {NumLessThan(TIME)}.','{NAME1} reached before {NAME2}.','contradiction'),
    make_template('{NAME1} reached {LOCATION} at {NumLessThan(TIME)} whereas {NAME2} reached at {TIME}.','{NAME1} reached after {NAME2}.','contradiction'),
    
    make_template('{NAME1} reached {LOCATION} at {TIME} whereas {NAME2} reached at {NumLessThan(TIME)}.','{NAME1} reached later than {NAME2}.','entailment'),
    make_template('{NAME1} reached {LOCATION} at {NumLessThan(TIME)} whereas {NAME2} reached at {TIME}.','{NAME1} reached earlier than {NAME2}.','entailment'),
    
    make_template('{NAME1} reached {LOCATION} at {TIME} whereas {NAME2} reached at {NumLessThan(TIME)}.','{NAME1} reached earlier than {NAME2}.','contradiction'),
    make_template('{NAME1} reached {LOCATION} at {NumLessThan(TIME)} whereas {NAME2} reached at {TIME}.','{NAME1} reached later than {NAME2}.','contradiction'),
    
    make_template('{NAME} has {SUBJECT1} class followed by {SUBJECT2} class.','{Capitalise(SUBJECT1)} class is before {SUBJECT2} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class followed by {SUBJECT2} class.','{Capitalise(SUBJECT2)} class is after {SUBJECT1} class.','entailment'),
    
    make_template('{NAME} has {SUBJECT1} class followed by {SUBJECT2} class.','{Capitalise(SUBJECT1)} class is after {SUBJECT2} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class followed by {SUBJECT2} class.','{Capitalise(SUBJECT2)} class is before {SUBJECT1} class.','contradiction'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class.','{Capitalise(SUBJECT1)} class is before {SUBJECT2} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class.','{Capitalise(SUBJECT2)} class is after {SUBJECT1} class.','entailment'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class.','{Capitalise(SUBJECT1)} class is after {SUBJECT2} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class.','{Capitalise(SUBJECT2)} class is before {SUBJECT1} class.','contradiction'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is before {SUBJECT2} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is before {SUBJECT3} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is before {SUBJECT3} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is after {SUBJECT1} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is after {SUBJECT1} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is after {SUBJECT2} class.','entailment'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is before {SUBJECT1} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is before {SUBJECT1} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is before {SUBJECT2} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is after {SUBJECT2} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is after {SUBJECT3} class.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is after {SUBJECT3} class.','contradiction'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is the first class among the three.','entailment'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is the last class among the three.','entailment'),
    
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is the first class among the three.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT3)} class is the first class among the three.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT1)} class is the last class among the three.','contradiction'),
    make_template('{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.','{Capitalise(SUBJECT2)} class is the last class among the three.','contradiction'),
    
    make_template('{NAME} has {SUBJECT1} class after {SUBJECT2} class.','{Capitalise(SUBJECT2)} class is before {SUBJECT1} class.','entailment'),
    make_template('{NAME} has {SUBJECT1} class after {SUBJECT2} class.','{Capitalise(SUBJECT1)} class is before {SUBJECT2} class.','contradiction'),
]

T['Spatial'] = [
    make_template('{NAME} was facing {DIRECTION} and turned to {POSSESSIVE_PRONOUN} {TURN}.','{NAME} is now facing {DirectionCorrect(DIRECTION,TURN)}.','entailment'),
    make_template('{NAME} was facing {DIRECTION} and turned to {POSSESSIVE_PRONOUN} {TURN}.','{NAME} is now facing {DirectionIncorrect(DIRECTION,TURN)}.','contradiction'),
    
    make_template('{CITY1} is {NumLessThan(MILE)} miles from {CITY2} and {MILE} miles from {CITY3}.','{CITY1} is nearer to {CITY2} than {CITY1} is to {CITY3}.','entailment'),
    make_template('{CITY1} is {NumLessThan(MILE)} miles from {CITY2} and {MILE} miles from {CITY3}.','{CITY1} is farther to {CITY3} than {CITY1} is to {CITY2}.','entailment'),
    
    make_template('{CITY1} is {MILE} miles from {CITY2} and {NumLessThan(MILE)} miles from {CITY3}.','{CITY1} is nearer to {CITY2} than {CITY1} is to {CITY3}.','contradiction'),
    make_template('{CITY1} is {MILE} miles from {CITY2} and {NumLessThan(MILE)} miles from {CITY3}.','{CITY1} is farther to {CITY3} than {CITY1} is to {CITY2}.','contradiction'),
    
    make_template('{CITY1} is located between {CITY2} and {CITY3}.','{CITY2} is nearer to {CITY1} than {CITY2} is to {CITY3}.','entailment'),
    make_template('{CITY1} is located between {CITY2} and {CITY3}.','{CITY3} is nearer to {CITY1} than {CITY3} is to {CITY2}.','entailment'),
    
    make_template('{CITY1} is located between {CITY2} and {CITY3}.','{CITY2} is nearer to {CITY3} than {CITY2} is to {CITY1}.','contradiction'),
    make_template('{CITY1} is located between {CITY2} and {CITY3}.','{CITY3} is nearer to {CITY2} than {CITY3} is to {CITY1}.','contradiction'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.','{NAME1} is to the left of {NAME3}.','entailment'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME2} is to the right of {NAME3}.','{NAME1} is to the right of {NAME3}.','entailment'),
    make_template('{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.','{NAME3} is to the right of {NAME1}.','entailment'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME2} is to the right of {NAME3}.','{NAME3} is to the left of {NAME1}.','entailment'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME1} is to the left of {NAME3}.','entailment'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME1} is to the right of {NAME3}.','entailment'),
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME3} is to the right of {NAME1}.','entailment'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME3} is to the left of {NAME1}.','entailment'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME2} is in between {NAME1} and {NAME3}.','entailment'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME2} is in between {NAME1} and {NAME3}.','entailment'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.','{NAME1} is to the right of {NAME3}.','contradiction'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME2} is to the right of {NAME3}.','{NAME1} is to the left of {NAME3}.','contradiction'),
    make_template('{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.','{NAME3} is to the left of {NAME1}.','contradiction'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME2} is to the right of {NAME3}.','{NAME3} is to the right of {NAME1}.','contradiction'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME1} is to the right of {NAME3}.','contradiction'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME1} is to the left of {NAME3}.','contradiction'),
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME3} is to the left of {NAME1}.','contradiction'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME3} is to the right of {NAME1}.','contradiction'),
    
    make_template('{NAME1} is to the left of {NAME2}. {NAME3} is to the right of {NAME2}.','{NAME1} is in between {NAME2} and {NAME3}.','contradiction'),
    make_template('{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.','{NAME3} is in between {NAME1} and {NAME2}.','contradiction'),

]

T['Coreference'] = [
    make_template('{NAME} has {A} {OBJECT}. {CAPITAL_PRONOUN} is {ADJECTIVE_OF_PERSON}.','{NAME} is {ADJECTIVE_OF_PERSON}.','entailment'),
    make_template('{NAME} has {A} {OBJECT}. {CAPITAL_PRONOUN} is {ADJECTIVE_OF_PERSON}.','{Capitalise(OBJECT)} is {ADJECTIVE_OF_PERSON}.','contradiction'),
    
    make_template('{NAME} has {A} {OBJECT}. It is {COLOUR} in colour.','{Capitalise(OBJECT)} is {COLOUR} in colour.','entailment'),
    make_template('{NAME} has {A} {OBJECT}. It is {COLOUR} in colour.','{NAME} is {COLOUR} in colour.','contradiction'),
    
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. He is {A} {PROFESSION1} and she is {A} {PROFESSION2}.','{MALE_NAME} is {A} {PROFESSION1}.', 'entailment'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. He is {A} {PROFESSION1} and she is {A} {PROFESSION2}.','{MALE_NAME} is {A} {PROFESSION2}.', 'contradiction'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. He is {A} {PROFESSION1} and she is {A} {PROFESSION2}.','{FEMALE_NAME} is {A} {PROFESSION1}.', 'contradiction'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. He is {A} {PROFESSION1} and she is {A} {PROFESSION2}.','{FEMALE_NAME} is {A} {PROFESSION2}.', 'entailment'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. She is {A} {PROFESSION1} and he is {A} {PROFESSION2}.','{MALE_NAME} is {A} {PROFESSION2}.', 'entailment'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. She is {A} {PROFESSION1} and he is {A} {PROFESSION2}.','{MALE_NAME} is {A} {PROFESSION1}.', 'contradiction'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. She is {A} {PROFESSION1} and he is {A} {PROFESSION2}.','{FEMALE_NAME} is {A} {PROFESSION2}.', 'contradiction'),
    make_template('{MALE_NAME} and {FEMALE_NAME} are friends. She is {A} {PROFESSION1} and he is {A} {PROFESSION2}.','{FEMALE_NAME} is {A} {PROFESSION1}.', 'entailment'),
    
    make_template('{MALE_NAME1} went with {MALE_NAME2} to the party. {MALE_NAME3} went with him to the match.','{MALE_NAME3} went with {MALE_NAME2} to the match.','entailment'),
    make_template('{MALE_NAME1} went with {MALE_NAME2} to the party. {MALE_NAME3} went with him to the match.','{MALE_NAME3} went with {MALE_NAME1} to the match.','contradiction'),
]

T['Relational'] = [
    make_template('{NAME} and {POSSESSIVE_PRONOUN} {T12_RELATION} went to {LOCATION}.','{NAME} went to {LOCATION} with a stranger.','contradiction'),
    make_template('{NAME1} is {T12_RELATION} of {NAME2}.','{NAME2} is {T12_RELATION_OPP} of {NAME1}.','entailment'),
    make_template('{NAME1} is {T12_RELATION} of {NAME2}.','{NAME2} is {T12_RELATION_MISMATCH} of {NAME1}.','contradiction'),
]

T['Causal'] = [
    make_template('{NAME} moved from {COUNTRY1} to {COUNTRY2}.','{NAME} now lives in {COUNTRY2}.','entailment'),
    make_template('{NAME} moved from {COUNTRY1} to {COUNTRY2}.','{NAME} now lives in {COUNTRY1}.','contradiction'),
    make_template('{NAME} moved to {COUNTRY1} from {COUNTRY2}.','{NAME} now lives in {COUNTRY1}.','entailment'),
    make_template('{NAME} moved to {COUNTRY1} from {COUNTRY2}.','{NAME} now lives in {COUNTRY2}.','contradiction'),
    
    make_template('{NAME} moved the {OBJECT_TABLE} from {LOCATION_HOUSE1} to {LOCATION_HOUSE2}.','The {OBJECT_TABLE} is now in {LOCATION_HOUSE2}.','entailment'),
    make_template('{NAME} moved the {OBJECT_TABLE} from {LOCATION_HOUSE1} to {LOCATION_HOUSE2}.','The {OBJECT_TABLE} is now in {LOCATION_HOUSE1}.','contradiction'),
    make_template('{NAME} moved the {OBJECT_TABLE} to {LOCATION_HOUSE1} from {LOCATION_HOUSE2}.','The {OBJECT_TABLE} is now in {LOCATION_HOUSE1}.','entailment'),
    make_template('{NAME} moved the {OBJECT_TABLE} to {LOCATION_HOUSE1} from {LOCATION_HOUSE2}.','The {OBJECT_TABLE} is now in {LOCATION_HOUSE2}.','contradiction'),
    
    make_template('{NAME1} {T6_CAUSAL_VERB} {T6_CAUSAL_OBJECT} to {NAME2}.','{NAME2} {T6_CAUSAL_VERB_OPP} {T6_CAUSAL_OBJECT} from {NAME1}.', 'entailment'),
    make_template('{NAME1} {T6_CAUSAL_VERB} {T6_CAUSAL_OBJECT} to {NAME2}.','{NAME1} {T6_CAUSAL_VERB_OPP} {T6_CAUSAL_OBJECT} from {NAME2}.', 'contradiction'),
]

T['Presupposition'] = [
    make_template('{NAME}\'s {T12_RELATION} is {ADJECTIVE_OF_PERSON}.','{NAME} has {A} {T12_RELATION}.','entailment'),
    make_template('{NAME}\'s {T12_RELATION} is {ADJECTIVE_OF_PERSON}.','{NAME} does not have {A} {T12_RELATION}.','contradiction'),
    
    make_template('{NAME}\'s {OBJECT} is {COLOUR} in colour.','{NAME} has {A} {OBJECT}.','entailment'),
    make_template('{NAME}\'s {OBJECT} is {COLOUR} in colour.','{NAME} does not have {A} {OBJECT}.','contradiction'),
    
    make_template('Is {NAME}\'s {T12_RELATION} {ADJECTIVE_OF_PERSON}.','{NAME} has {A} {T12_RELATION}.','entailment'),
    make_template('Is {NAME}\'s {T12_RELATION} {ADJECTIVE_OF_PERSON}.','{NAME} does not have {A} {T12_RELATION}.','contradiction'),
    
    make_template('Is {NAME}\'s {OBJECT} {COLOUR} in colour.','{NAME} has {A} {OBJECT}.','entailment'),
    make_template('Is {NAME}\'s {OBJECT} {COLOUR} in colour.','{NAME} does not have {A} {OBJECT}.','contradiction'),
    
    make_template('All {N} {OBJECTS} are {COLOUR} in colour.','Exactly {N} {OBJECTS} are {COLOUR} in colour.','entailment'),
    
    make_template('{NAME1} {T7_VERB_PAST} {NAME2}.','{NAME1} {T7_VERB_PAST} someone.','entailment'),
    make_template('{NAME1} {T7_VERB_PAST} {NAME2}.','Someone {T7_VERB_PAST} {NAME2}.','entailment'),
    
    make_template('{NAME1} {T7_VERB_PAST} {NAME2}.','{NAME1} did not {T7_VERB} someone.','contradiction'),
    
    make_template('{NAME1} {T7_VERB_PAST} {NAME2}.','{NAME1} {T7_VERB_PAST} no one.','contradiction'),
    make_template('{NAME1} {T7_VERB_PAST} {NAME2}.','No one {T7_VERB_PAST} {NAME2}.','contradiction'),
    
    make_template('{NAME} has stopped {T8_VERB_CONTINUOUS}.','{NAME} used to {T8_VERB}.', 'entailment'),
    
    make_template('{NAME} returned to {COUNTRY}.','{NAME} has been to {COUNTRY} before.','entailment'),
    make_template('{NAME} returned to {COUNTRY}.','{NAME} has never been to {COUNTRY} before.','contradiction'),
    
    make_template('If {NAME} was {A} {T10_PROFESSION}, {PRONOUN} would {T10_ACTION}.','{NAME} is {A} {T10_PROFESSION}.','contradiction'),
    make_template('If {NAME} was {A} {T10_PROFESSION}, {PRONOUN} would {T10_ACTION}.','{NAME} is not {A} {T10_PROFESSION}.','entailment'),
    
    make_template('{NAME} was {EMOTION} when {PRONOUN} {T9_PAST}.','{NAME} {T9_PAST}.','entailment'),
    make_template('I {KNOW} that {NAME} {T9_PAST}.','{NAME} {T9_PAST}.','entailment'),
    make_template('{NAME} has talked to every {SCHOOL_NOUN} in {POSSESSIVE_PRONOUN} {SCHOOL_ENTITY}.','There are {SCHOOL_NOUN} in {NAME}\'s {SCHOOL_ENTITY}.','entailment'),
]

T['Implicature'] = [
    make_template('Some of the {OBJECTS} are {COLOUR} in colour.','All of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('Some of the {OBJECTS} are {COLOUR} in colour.','Not all of the {OBJECTS} are {COLOUR} in colour.','entailment'),
    make_template('Not all of the {OBJECTS} are {COLOUR} in colour.','Some of the {OBJECTS} are {COLOUR} in colour.','entailment'),
    make_template('All of the {OBJECTS} are {COLOUR} in colour.','Some of the {OBJECTS} are {COLOUR} in colour.','contradiction'),
    make_template('{NAME} may {VERB}.','{NAME} will {VERB}.','contradiction'),
    make_template('{NAME} has {N} {OBJECTS}.','{NAME} has no more than {N} {OBJECTS}.','entailment'),
    
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE1}.','{NAME} also asked for {OBJECT_TABLE2}.','contradiction'),
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE2}.','{NAME} also asked for {OBJECT_TABLE1}.','contradiction'),
    
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE1}.','{NAME} did not ask for {OBJECT_TABLE2}.','entailment'),
    make_template('{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE2}.','{NAME} did not ask for {OBJECT_TABLE1}.','entailment'),
    
    make_template('{NAME} {T11_ACTION}.','It is {NAME}\'s {T11_OBJECT}.','contradiction'),
    make_template('{NAME} {T11_ACTION}.','It is not {NAME}\'s {T11_OBJECT}.','entailment'),
    
    make_template('{NAME} is meeting a {T12_RELATION_GENDER} today.','The {T12_RELATION_GENDER} is not {NAME}\'s {T12_RELATION}.','entailment'),
    make_template('{NAME} is meeting a {T12_RELATION_GENDER} today.','The {T12_RELATION_GENDER} is {NAME}\'s {T12_RELATION}.','contradiction'),
    
    make_template('Even {NAME} {T9_PAST}.','{NAME} was among the least likely to {T9_PRESENT}.','entailment'),
    make_template('Even {NAME} {T9_PAST}.','{NAME} was among the most likely to {T9_PRESENT}.','contradiction'),
    
    make_template('{NAME1} asked {NAME2} for {DOLLAR} dollars. {NAME2} had {NumLessThan(DOLLAR)}.','{NAME2} had {DOLLAR} dollars.','contradiction'),
    make_template('{NAME1} asked {NAME2} for {DOLLAR} dollars. {NAME2} had {NumLessThan(DOLLAR)}.','{NAME2} did not have {DOLLAR} dollars.','entailment'),
    
    make_template('{NAME1} asked {NAME2} for {DOLLAR} dollars. {NAME2} refused.','{NAME1} has higher chances of convincing {NAME2} for {NumLessThan(DOLLAR)} dollars.','entailment'),
    make_template('{NAME1} asked {NAME2} for {NumLessThan(DOLLAR)} dollars. {NAME2} refused.','{NAME1} has higher chances of convincing {NAME2} for {DOLLAR} dollars.','contradiction'),
]

T['World'] = [
    make_template('{NAME} lives in {T4_CAPITAL}.','{NAME} lives in {T4_COUNTRY}.','entailment'),
    make_template('{NAME} lives in {T4_CAPITAL1}.','{NAME} lives in {T4_COUNTRY2}.','contradiction'),
]

T['Taxonomic'] = [
    make_template('All {T13_CLASS} {T13_PROPERTY}.','{T13_ELEMENT} {T13_PROPERTY}.','entailment'),
    make_template('All {T13_CLASS} {T13_PROPERTY}.','{T13_ELEMENT} don\'t {T13_PROPERTY}.','contradiction'),
    make_template('Some {T13_CLASS} {T13_PROPERTY}.','{T13_ELEMENT} {T13_PROPERTY}.','neutral'),
]

Capabilities = ['Lexical','Syntactic','Negation','Boolean','Comparative','Quantifier','Conditional','Numerical',
                'Coreference','Spatial','Temporal','Causal','Relational','Taxonomic','World','Presupposition',
                'Implicature']

c = 0
for capability in Capabilities:
    print("| %-16s | %3d |"%(capability, len(T[capability])))
    c += len(T[capability])
    
print ("| %-16s | %3d |"%('Total', c))

| Lexical          |  12 |
| Syntactic        |   9 |
| Negation         |   8 |
| Boolean          |  62 |
| Comparative      |  54 |
| Quantifier       |  16 |
| Conditional      |   6 |
| Numerical        |  32 |
| Coreference      |  14 |
| Spatial          |  30 |
| Temporal         |  60 |
| Causal           |  10 |
| Relational       |   3 |
| Taxonomic        |   3 |
| World            |   2 |
| Presupposition   |  22 |
| Implicature      |  20 |
| Total            | 363 |


In [6]:
random.seed(0)

NUM_EXAMPLES = 1000
FOLDER = 'final'

os.makedirs(FOLDER, exist_ok=True)

for cp in Capabilities:
    for i, tm in enumerate(T[cp]):
        examples = []
        for idx in range(NUM_EXAMPLES):
            tm_copy = copy.deepcopy(tm)
            filled_tm = fill_template(tm_copy)
            examples.append(filled_tm)
        save_file(examples, os.path.join(FOLDER, cp.lower() + '-' + str(i+1) + '.tsv'))
        print (cp.lower() + '-' + str(i+1))
        print (T[cp][i])
        print (examples[:1])

lexical-1
{'P': '{NAME} is {T0_ADJ}.', 'H': '{NAME} is {T0_SYN}.', 'label': 'entailment'}
[{'P': 'Al is transparent.', 'H': 'Al is diaphanous.', 'label': 'entailment'}]
lexical-2
{'P': '{NAME} is {T0_ADJ}.', 'H': '{NAME} is {T0_ANT}.', 'label': 'contradiction'}
[{'P': 'John is brittle.', 'H': 'John is tough.', 'label': 'contradiction'}]
lexical-3
{'P': '{NAME} is {ADJECTIVE_OF_PERSON}.', 'H': '{NAME} lives in {COUNTRY}.', 'label': 'neutral'}
[{'P': 'Janet is dull.', 'H': 'Janet lives in Taiwan.', 'label': 'neutral'}]
lexical-4
{'P': '{NAME} is {ADJECTIVE_OF_PERSON}.', 'H': '{NAME} is {A} {PROFESSION}.', 'label': 'neutral'}
[{'P': 'Dave is drowsy.', 'H': 'Dave is a dentist.', 'label': 'neutral'}]
lexical-5
{'P': '{NAME} lives in {COUNTRY}.', 'H': '{NAME} is {ADJECTIVE_OF_PERSON}.', 'label': 'neutral'}
[{'P': 'Marie lives in Iran.', 'H': 'Marie is saucy.', 'label': 'neutral'}]
lexical-6
{'P': '{NAME} lives in {COUNTRY}.', 'H': '{NAME} is {A} {PROFESSION}.', 'label': 'neutral'}
[{'P': 'Su

boolean-9
{'P': '{NAME1}, {NAME2} and {NAME3} are {ADJECTIVE_OF_PERSON}.', 'H': '{NAME2} is {ADJECTIVE_OF_PERSON}.', 'label': 'entailment'}
[{'P': 'Tim, Bob and Susan are evasive.', 'H': 'Bob is evasive.', 'label': 'entailment'}]
boolean-10
{'P': '{NAME1}, {NAME2} and {NAME3} are {ADJECTIVE_OF_PERSON}.', 'H': '{NAME3} is {ADJECTIVE_OF_PERSON}.', 'label': 'entailment'}
[{'P': 'Edith, Ralph and Jack are balanced.', 'H': 'Jack is balanced.', 'label': 'entailment'}]
boolean-11
{'P': '{NAME1} or {NAME2} are {ADJECTIVE_OF_PERSON}.', 'H': '{NAME1} is {ADJECTIVE_OF_PERSON}.', 'label': 'neutral'}
[{'P': 'Karen or Colin are well-developed.', 'H': 'Karen is well-developed.', 'label': 'neutral'}]
boolean-12
{'P': '{NAME1} or {NAME2} are {ADJECTIVE_OF_PERSON}.', 'H': '{NAME2} is {ADJECTIVE_OF_PERSON}.', 'label': 'neutral'}
[{'P': 'Jimmy or Ralph are affectionate.', 'H': 'Ralph is affectionate.', 'label': 'neutral'}]
boolean-13
{'P': '{NAME} is {ADJECTIVE_OF_PERSON1} and {ADJECTIVE_OF_PERSON2}.', 'H

boolean-37
{'P': '{NAME1}, {NAME2}, {NAME3} and {NAME4} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3} and {COUNTRY4} respectively.', 'H': '{NAME4} is from {COUNTRY1}.', 'label': 'contradiction'}
[{'P': 'Philip, Heather, Angela and Paul are from Cuba, Azerbaijan, Vietnam and Netherlands respectively.', 'H': 'Paul is from Cuba.', 'label': 'contradiction'}]
boolean-38
{'P': '{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.', 'H': '{NAME1} is from {COUNTRY1}.', 'label': 'entailment'}
[{'P': 'Sam, Cynthia, Rebecca, Lauren and Don are from Israel, Uruguay, Nepal, France and Ukraine respectively.', 'H': 'Sam is from Israel.', 'label': 'entailment'}]
boolean-39
{'P': '{NAME1}, {NAME2}, {NAME3}, {NAME4} and {NAME5} are from {COUNTRY1}, {COUNTRY2}, {COUNTRY3}, {COUNTRY4} and {COUNTRY5} respectively.', 'H': '{NAME2} is from {COUNTRY2}.', 'label': 'entailment'}
[{'P': 'Kathryn, Mark, Robert, Alex and Henry are from Uk

boolean-61
{'P': '{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE1}.', 'H': '{NAME} also asked for {OBJECT_TABLE2}.', 'label': 'neutral'}
[{'P': 'Fork and knife lie on table. Alexandra asked for fork.', 'H': 'Alexandra also asked for knife.', 'label': 'neutral'}]
boolean-62
{'P': '{Capitalise(OBJECT_TABLE1)} and {OBJECT_TABLE2} lie on table. {NAME} asked for {OBJECT_TABLE2}.', 'H': '{NAME} also asked for {OBJECT_TABLE1}.', 'label': 'neutral'}
[{'P': 'Perfume and pin lie on table. Walter asked for pin.', 'H': 'Walter also asked for perfume.', 'label': 'neutral'}]
comparative-1
{'P': '{NAME1} is {T1_COM} than {NAME2}.', 'H': '{NAME1} and {NAME2} can be compared in terms of how {T1_POS} they are.', 'label': 'entailment'}
[{'P': 'Sophie is smarter than Jimmy.', 'H': 'Sophie and Jimmy can be compared in terms of how smart they are.', 'label': 'entailment'}]
comparative-2
{'P': '{NAME1} is {T2_COM} than {NAME2}.', 'H': '{NAME1} and {NAME2} can be 

comparative-29
{'P': 'Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME2}.', 'H': '{NAME3} is {T1_COM} than {NAME2}.', 'label': 'contradiction'}
[{'P': 'Among Sally, Helen and Colin the sincerest is Helen.', 'H': 'Colin is sincerer than Helen.', 'label': 'contradiction'}]
comparative-30
{'P': 'Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.', 'H': '{NAME1} is {T1_COM} than {NAME3}.', 'label': 'contradiction'}
[{'P': 'Among Dave, Samuel and Scott the proudest is Scott.', 'H': 'Dave is prouder than Scott.', 'label': 'contradiction'}]
comparative-31
{'P': 'Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {NAME3}.', 'H': '{NAME2} is {T1_COM} than {NAME3}.', 'label': 'contradiction'}
[{'P': 'Among Edwin, Janet and Christine the cleverest is Christine.', 'H': 'Janet is cleverer than Christine.', 'label': 'contradiction'}]
comparative-32
{'P': '{NAME1} is {T1_COM} than {NAME2}. {NAME1} is {T1_COM} than {NAME3}.', 'H': 'Among {NAME1}, {NAME2} and {NAME3} the {T1_SUP} is {N

...

spatial-20
{'P': '{NAME1} is to the right of {NAME2}. {NAME3} is to the left of {NAME2}.', 'H': '{NAME2} is in between {NAME1} and {NAME3}.', 'label': 'entailment'}
[{'P': 'Ann is to the right of Laura. Alex is to the left of Laura.', 'H': 'Laura is in between Ann and Alex.', 'label': 'entailment'}]
spatial-21
{'P': '{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.', 'H': '{NAME1} is to the right of {NAME3}.', 'label': 'contradiction'}
[{'P': 'Ray is to the left of Kathy. Kathy is to the left of James.', 'H': 'Ray is to the right of James.', 'label': 'contradiction'}]
spatial-22
{'P': '{NAME1} is to the right of {NAME2}. {NAME2} is to the right of {NAME3}.', 'H': '{NAME1} is to the left of {NAME3}.', 'label': 'contradiction'}
[{'P': 'Sandra is to the right of Carl. Carl is to the right of Jason.', 'H': 'Sandra is to the left of Jason.', 'label': 'contradiction'}]
spatial-23
{'P': '{NAME1} is to the left of {NAME2}. {NAME2} is to the left of {NAME3}.', 'H': '{NAME3}

temporal-20
{'P': '{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.', 'H': '{NAME1} was born earlier than {NAME2}.', 'label': 'entailment'}
[{'P': 'Lynn was born in 1995 and Anthony was born in 1997.', 'H': 'Lynn was born earlier than Anthony.', 'label': 'entailment'}]
temporal-21
{'P': '{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.', 'H': '{NAME1} was born earlier than {NAME2}.', 'label': 'contradiction'}
[{'P': 'Alison was born in 1995 and Sophie was born in 1993.', 'H': 'Alison was born earlier than Sophie.', 'label': 'contradiction'}]
temporal-22
{'P': '{NAME1} was born in {NumLessThan(YEAR)} and {NAME2} was born in {YEAR}.', 'H': '{NAME2} was born earlier than {NAME1}.', 'label': 'contradiction'}
[{'P': 'Deborah was born in 1990 and Jimmy was born in 1991.', 'H': 'Jimmy was born earlier than Deborah.', 'label': 'contradiction'}]
temporal-23
{'P': '{NAME1} was born in {YEAR} and {NAME2} was born in {NumLessThan(YEAR)}.', 'H': '{NAME2

temporal-49
{'P': '{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.', 'H': '{Capitalise(SUBJECT3)} class is before {SUBJECT2} class.', 'label': 'contradiction'}
[{'P': 'Ryan has literature class then latin class then computer science class.', 'H': 'Computer science class is before latin class.', 'label': 'contradiction'}]
temporal-50
{'P': '{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.', 'H': '{Capitalise(SUBJECT1)} class is after {SUBJECT2} class.', 'label': 'contradiction'}
[{'P': 'Harry has geology class then political science class then music class.', 'H': 'Geology class is after political science class.', 'label': 'contradiction'}]
temporal-51
{'P': '{NAME} has {SUBJECT1} class then {SUBJECT2} class then {SUBJECT3} class.', 'H': '{Capitalise(SUBJECT1)} class is after {SUBJECT3} class.', 'label': 'contradiction'}
[{'P': 'Lauren has chemistry class then art class then physics class.', 'H': 'Chemistry class is after physics class.', 

world-2
{'P': '{NAME} lives in {T4_CAPITAL1}.', 'H': '{NAME} lives in {T4_COUNTRY2}.', 'label': 'contradiction'}
[{'P': 'Jerry lives in Hanoi.', 'H': 'Jerry lives in Netherlands.', 'label': 'contradiction'}]
presupposition-1
{'P': "{NAME}'s {T12_RELATION} is {ADJECTIVE_OF_PERSON}.", 'H': '{NAME} has {A} {T12_RELATION}.', 'label': 'entailment'}
[{'P': "Jimmy's father is warmhearted.", 'H': 'Jimmy has a father.', 'label': 'entailment'}]
presupposition-2
{'P': "{NAME}'s {T12_RELATION} is {ADJECTIVE_OF_PERSON}.", 'H': '{NAME} does not have {A} {T12_RELATION}.', 'label': 'contradiction'}
[{'P': "Alice's daughter is frank.", 'H': 'Alice does not have a daughter.', 'label': 'contradiction'}]
presupposition-3
{'P': "{NAME}'s {OBJECT} is {COLOUR} in colour.", 'H': '{NAME} has {A} {OBJECT}.', 'label': 'entailment'}
[{'P': "Fiona's battery is black in colour.", 'H': 'Fiona has a battery.', 'label': 'entailment'}]
presupposition-4
{'P': "{NAME}'s {OBJECT} is {COLOUR} in colour.", 'H': '{NAME} does

implicature-13
{'P': '{NAME} is meeting a {T12_RELATION_GENDER} today.', 'H': "The {T12_RELATION_GENDER} is not {NAME}'s {T12_RELATION}.", 'label': 'entailment'}
[{'P': 'Julia is meeting a man today.', 'H': "The man is not Julia's husband.", 'label': 'entailment'}]
implicature-14
{'P': '{NAME} is meeting a {T12_RELATION_GENDER} today.', 'H': "The {T12_RELATION_GENDER} is {NAME}'s {T12_RELATION}.", 'label': 'contradiction'}
[{'P': 'Kathleen is meeting a woman today.', 'H': "The woman is Kathleen's daughter.", 'label': 'contradiction'}]
implicature-15
{'P': 'Even {NAME} {T9_PAST}.', 'H': '{NAME} was among the least likely to {T9_PRESENT}.', 'label': 'entailment'}
[{'P': 'Even Bob got married.', 'H': 'Bob was among the least likely to get married.', 'label': 'entailment'}]
implicature-16
{'P': 'Even {NAME} {T9_PAST}.', 'H': '{NAME} was among the most likely to {T9_PRESENT}.', 'label': 'contradiction'}
[{'P': 'Even Richard shifted to Europe.', 'H': 'Richard was among the most likely to shi